In [1]:
# 常规sklearn相关的库和包
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np
import os

# 常规的PyTorch相关的库和包
import torch
from torch import nn
from torch.nn import functional as F

# PyTorch 数据打包相关
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
save_path =  "r./saved_weights2" # r 不改变里面的转义字符
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [3]:
"""
    生成和切分数据
"""
n_features = 1024

# 生成数据
X, y = make_regression(n_samples=100000, n_features=n_features, n_informative=50,random_state= 0)
X += np.random.randn(*X.shape) * 0.001 # *号可以自动解包

# 切分数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state= 0)

In [4]:
#print(*X.shape)

In [5]:
#X.shape

In [6]:
#np.random.randn(*X.shape)

In [7]:
# a = (1,2,3) 
# def f(b,c,d): 
#     pass 

# f(*a) # b = 1, c = 2, d = 3

"""
    深度学习实现线性回归
    
        - 数据打包
        - 模型搭建
        - 模型训练
        - 模型评估
"""

In [8]:
"""
    数据打包
"""

class MyDataset(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __getitem__(self, index):
        return torch.tensor(data=self.X[index], dtype=torch.float32), torch.tensor(data=[self.y[index]], dtype=torch.float32)
    
    def __len__(self):
        return len(self.y)

# 训练集数据加载器
train_dataset = MyDataset(X=X_train, y=y_train)

# print(train_dataset.__len__())

# print(train_dataset.__getitem__(0))

train_dataloader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)

# 测试集数据加载器
test_dataset = MyDataset(X=X_test, y=y_test)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=512, shuffle=False)

In [9]:
# 检测设备

# device = "cuda:0" if torch.cuda.is_available() else "cpu"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
"""
    定义模型
"""

class Model(nn.Module):
    
    def __init__(self, in_features, out_features):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(in_features=in_features, out_features=512)
        self.linear2 = nn.Linear(in_features=512, out_features=128)
        self.linear3 = nn.Linear(in_features=128, out_features=out_features)
    
    def forward(self, x):
        
        x = self.linear1(x)
        x = F.relu(x)
        
        x = self.linear2(x)
        x = F.relu(x)
        
        x = self.linear3(x)
        return x
    

In [11]:
# 构建模型
model = Model(in_features=n_features, out_features=1)

# 参数按照实际设备类型搬家
model.to(device=device)

Model(
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=1, bias=True)
)

In [12]:
# 构建损失函数
loss_fn = nn.MSELoss()

In [13]:
# 构建优化器
optimizer = torch.optim.SGD(params=model.parameters(), lr=5e-5)

In [14]:
# 过程监控相关的工具
def get_loss(model=model, dataloader=test_dataloader, loss_fn=loss_fn):
    
    model.eval()
    
    with torch.no_grad():
        # 累计每个批量的损失
        losses = []
        for X, y in dataloader:
            
            # 数据搬家
            X = X.to(device=device)
            y = y.to(device=device)
            
            # 正向传播
            y_pred = model(X)
            
            # 计算损失
            loss = loss_fn(y_pred, y)
            
            # 追加到结果列表中
            losses.append(loss)
                     
        return torch.tensor(data=losses, dtype=torch.float32).mean().item()
    

In [15]:
# 定义训练过程

def train(model=model, 
          epochs=20, 
          loss_fn=loss_fn, 
          optimizer=optimizer, 
          train_dataloader=train_dataloader,
          test_dataloader=test_dataloader):
    
    best_loss = np.inf
    for epoch in range(1, epochs + 1):
        
        for X, y in train_dataloader:
            
            # 数据搬家
            X = X.to(device=device)
            y = y.to(device=device)
            
            # 正向传播
            y_pred = model(X)
            
            # 计算损失
            loss = loss_fn(y_pred, y)
            
            # 梯度下降
            loss.backward()
            
            # 优化一步
            optimizer.step()
            
            # 清空梯度
            optimizer.zero_grad()
            
        # 过程监控
        # 按epoch 来
        # 整个训练集上的情况，整个测试集上的情况
        train_loss = round(get_loss(model=model, dataloader=train_dataloader, loss_fn=loss_fn), 3)
        test_loss = round(get_loss(model=model, dataloader=test_dataloader, loss_fn=loss_fn), 3)
                    
        print("当前为第 {} 轮，训练集损失为：{}，测试集损失为：{}".format(epoch, train_loss, test_loss))
        
        if test_loss < best_loss:
            print("保存模型")
            best_loss = test_loss
            torch.save(obj=model.state_dict(), f=os.path.join(save_path, "epoch_{}_train_loss_{}_test_acc_{}.pt".format(epoch, train_loss, test_loss)))

In [16]:
train()

当前为第 1 轮，训练集损失为：415.088，测试集损失为：413.48
保存模型
当前为第 2 轮，训练集损失为：178.317，测试集损失为：233.689
保存模型
当前为第 3 轮，训练集损失为：82.21，测试集损失为：151.942
保存模型
当前为第 4 轮，训练集损失为：45.104，测试集损失为：120.034
保存模型
当前为第 5 轮，训练集损失为：29.762，测试集损失为：106.168
保存模型
当前为第 6 轮，训练集损失为：20.642，测试集损失为：97.193
保存模型
当前为第 7 轮，训练集损失为：17.401，测试集损失为：95.292
保存模型
当前为第 8 轮，训练集损失为：12.118，测试集损失为：89.78
保存模型
当前为第 9 轮，训练集损失为：8.849，测试集损失为：85.665
保存模型
当前为第 10 轮，训练集损失为：7.437，测试集损失为：84.545
保存模型
当前为第 11 轮，训练集损失为：6.173，测试集损失为：83.378
保存模型
当前为第 12 轮，训练集损失为：5.186，测试集损失为：81.127
保存模型
当前为第 13 轮，训练集损失为：4.262，测试集损失为：80.811
保存模型
当前为第 14 轮，训练集损失为：3.9，测试集损失为：80.399
保存模型
当前为第 15 轮，训练集损失为：3.074，测试集损失为：79.7
保存模型
当前为第 16 轮，训练集损失为：2.69，测试集损失为：79.186
保存模型
当前为第 17 轮，训练集损失为：2.411，测试集损失为：78.472
保存模型
当前为第 18 轮，训练集损失为：2.205，测试集损失为：77.891
保存模型
当前为第 19 轮，训练集损失为：1.786，测试集损失为：77.713
保存模型
当前为第 20 轮，训练集损失为：1.881，测试集损失为：77.642
保存模型
